In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [39]:
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

In [40]:
n_input = 784
n_class = 10
dropout = 0.75

In [41]:
x = tf.placeholder(tf.float32,[None,n_input])
y = tf.placeholder(tf.float32,[None,n_class])
keep_prob = tf.placeholder(tf.float32)

In [42]:
#卷积操作
def conv2d(name,x,W,b,strides=1):
    x = tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding = 'SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x,name=name)
#池化操作
def maxpool2d(name,x,k=2):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding ='SAME',name=name)
#规范化操作
def norm(name,l_input,lsize=4):
    return tf.nn.lrn(l_input,lsize,bias=1.,alpha=0.001/9.0,beta=0.75,name=name)


In [56]:
weights = {
    'wc1':tf.Variable(tf.random_normal([11,11,1,96])),
    'wc2':tf.Variable(tf.random_normal([5,5,96,256])),
    'wc3':tf.Variable(tf.random_normal([3,3,256,384])),
    'wc4':tf.Variable(tf.random_normal([3,3,384,384])),
    'wc5':tf.Variable(tf.random_normal([3,3,384,256])),
    'wd1':tf.Variable(tf.random_normal([2*2*256,4096])),
    'wd2':tf.Variable(tf.random_normal([4096,4096])),
    'out':tf.Variable(tf.random_normal([4096,10]))
}

biases = {
    'bc1':tf.Variable(tf.random_normal([96])),
    'bc2':tf.Variable(tf.random_normal([256])),
    'bc3':tf.Variable(tf.random_normal([384])),
    'bc4':tf.Variable(tf.random_normal([384])),
    'bc5':tf.Variable(tf.random_normal([256])),
    'bd1':tf.Variable(tf.random_normal([4096])),
    'bd2':tf.Variable(tf.random_normal([4096])),
    'out':tf.Variable(tf.random_normal([10]))
}

In [57]:
def alex_net(x,weights,biases,dropout):
    x = tf.reshape(x,[-1,28,28,1])
    
    conv1 = conv2d('conv1',x,weights['wc1'],biases['bc1'])
    pool1 = maxpool2d('pool1',conv1,2)
    norm1 = norm('norm1',pool1,lsize=4)
    
    conv2 = conv2d('conv2',norm1,weights['wc2'],biases['bc2'])
    pool2 = maxpool2d('pool2',conv2,2)
    norm2 = norm('norm2',pool2,lsize=4)
    
    conv3 = conv2d('conv3', norm2, weights['wc3'], biases['bc3'])
    pool3 = maxpool2d('pool3', conv3, k=2)
    norm3 = norm('norm3', pool3, lsize=4)
    
    conv4 = conv2d('conv4', norm3, weights['wc4'], biases['bc4'])
    conv5 = conv2d('conv5', conv4, weights['wc5'], biases['bc5'])
    pool5 = maxpool2d('pool5', conv5, k=2)
    norm5 = norm('norm5', pool5, lsize=4)
    
    fc1 = tf.reshape(norm5,[-1,2*2*256])
    fc1 = tf.add(tf.matmul(fc1,weights['wd1']),biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1,dropout)
    
    
    fc2 = tf.add(tf.matmul(fc1,weights['wd2']),biases['bd2'])
    fc2 = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2,dropout)
    
    out = tf.add(tf.matmul(fc2,weights['out']),biases['out'])
    return out

In [58]:
pred = alex_net(x,weights,biases,keep_prob)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step*batch_size<training_iters:
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer,feed_dict={x:batch_x,y:batch_y,keep_prob:dropout})
        
        if step%display_step == 0:
            loss,acc = sess.run([cost,accuracy],feed_dict={x:batch_x,y:batch_y,keep_prob:1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss=,", loss)
            print('Training Accuracy= ',acc)
        
        step+=1
    print('Optimization Finished')
    
    print('Testing Accuracy:',sess.run(accuracy,feed_dict={x:mnist.test.images[:256],
                                                           y:mnist.test.labels[:256],
                                                           keep_prob:1.}))
            

Iter 1280, Minibatch Loss=, 35227.4
Training Accuracy=  0.28125
Iter 2560, Minibatch Loss=, 19786.7
Training Accuracy=  0.421875
Iter 3840, Minibatch Loss=, 9594.99
Training Accuracy=  0.648438
Iter 5120, Minibatch Loss=, 8298.59
Training Accuracy=  0.742188
Iter 6400, Minibatch Loss=, 2807.21
Training Accuracy=  0.867188
Iter 7680, Minibatch Loss=, 5230.12
Training Accuracy=  0.851562
Iter 8960, Minibatch Loss=, 3012.27
Training Accuracy=  0.882812
Iter 10240, Minibatch Loss=, 2697.16
Training Accuracy=  0.914062
Iter 11520, Minibatch Loss=, 4105.21
Training Accuracy=  0.859375
Iter 12800, Minibatch Loss=, 2920.83
Training Accuracy=  0.882812
Iter 14080, Minibatch Loss=, 5822.77
Training Accuracy=  0.84375
Iter 15360, Minibatch Loss=, 3192.21
Training Accuracy=  0.890625
Iter 16640, Minibatch Loss=, 3737.85
Training Accuracy=  0.898438
Iter 17920, Minibatch Loss=, 2315.04
Training Accuracy=  0.9375
Iter 19200, Minibatch Loss=, 1499.62
Training Accuracy=  0.953125
Iter 20480, Minibatch

In [54]:
batch_y.shape

(128, 10)